In [1]:
import pandas as pd
import numpy as np
import os

## Set Path

In [6]:
LF_result_path = ""
LF_export_path = ""
HY_result_path = "/Users/huiziyu/Library/CloudStorage/GoogleDrive-huiziy@g.ucla.edu/My Drive/Project - LLM in Biomedical & Health/new/results"
HY_export_path = "/Users/huiziyu/Library/CloudStorage/GoogleDrive-huiziy@g.ucla.edu/My Drive/Project - LLM in Biomedical & Health/new/results"
HY_import_path = "/Users/huiziyu/Library/CloudStorage/GoogleDrive-huiziy@g.ucla.edu/My Drive/Project - LLM Bib in Biomedical & Health/data/processed"

In [7]:
result_path = HY_result_path
data_export_path = HY_export_path
data_import_path = HY_import_path

In [9]:
df = pd.read_csv(data_import_path+"/relevant_data.csv")

In [11]:
# replace na with ""
df.fillna("",inplace=True)

In [12]:
abstract_df = df[['display_name','doi','abstract_text']].reset_index(drop=True)

In [13]:
# We use title and abstract, as well focus on sentence-line context aware info, not keywords.
abstract_df["Title_abstract"] = abstract_df["display_name"]+". "+abstract_df["abstract_text"]
# now titel only
# abstract_df["Title_abstract"] = abstract_df["display_name"]

### Topic Modeling

In [20]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans

In [25]:
from bertopic import BERTopic

ImportError: cannot import name 'mmr' from 'bertopic.representation._mmr' (/Users/huiziyu/opt/anaconda3/lib/python3.9/site-packages/bertopic/representation/_mmr.py)

In [24]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 MB 17.2 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1 requires torch==1.12.1, but you have torch 2.1.1 which is incompatible.
farm-haystack 1.10.0 requires torch<1.13,>1.9, but you have torch 2.1.1 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [33]:
docs = [str(item) for item in list(abstract_df.Title_abstract)]

## Add additional stopwords based on the query

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Your additional stop words
additional_stop_words = {"large language model", "GPT", "ChatGPT", "Llama", "Google Palm", "Anthropic Claude", "Large Language Model", "LLM", "Chatgpt","health", "medical", "Health", "Medical"}

# Combine default English stop words with your additional ones
custom_stop_words = list(ENGLISH_STOP_WORDS.union(additional_stop_words))

In [35]:
# [TODO 1] need to consider remove query keywords
# [TODO 2] remove non-English
# [TODO 3] cluster number: more experiments [20, 30, 40, 50, 100, 150, 200]
# 200 too many
vectorizer_model = CountVectorizer(stop_words=custom_stop_words)
# hdbscan_model = HDBSCAN(min_cluster_size=10,
#                         min_samples=5,
#                         metric='euclidean',
#                         cluster_selection_method='eom',
#                         prediction_data=True) # random_state=42
cluster_model = KMeans(n_clusters=200)
topic_model = BERTopic(hdbscan_model=cluster_model, #hdbscan_model,
                       vectorizer_model=vectorizer_model,
                       top_n_words=10) # nr_topics=100,
topics, probs = topic_model.fit_transform(docs)

In [36]:
topic_model.save(result_path+"/LLM_en_title_200")

In [37]:
abstract_df["topics"] = topics

In [38]:
abstract_df

,display_name,doi,abstract_text,Title_abstract,topics
0,Performance of ChatGPT on USMLE: Potential for...,https://doi.org/10.1371/journal.pdig.0000198,We evaluated the performance of a large langua...,Performance of ChatGPT on USMLE: Potential for...,35
1,How Does ChatGPT Perform on the United States ...,https://doi.org/10.2196/45312,Background Chat Generative Pre-trained Transfo...,How Does ChatGPT Perform on the United States ...,6
2,ChatGPT and the Future of Medical Writing,https://doi.org/10.1148/radiol.223312,"HomeRadiologyVol. 307, No. 2 PreviousNext Revi...",ChatGPT and the Future of Medical Writing,60
3,Performance of ChatGPT on USMLE: Potential for...,https://doi.org/10.1101/2022.12.19.22283643,ABSTRACT We evaluated the performance of a lar...,Performance of ChatGPT on USMLE: Potential for...,35
4,"The Role of ChatGPT, Generative Language Model...",https://doi.org/10.2196/46885,ChatGPT is a generative language model tool la...,"The Role of ChatGPT, Generative Language Model...",35
...,...,...,...,...,...
3412,Revolutionizing clinical experimental protocol...,https://doi.org/10.1016/j.amjms.2023.09.004,ChatGPT is a conversational artificial intelli...,Revolutionizing clinical experimental protocol...,28
3413,Applying GPT-4 to the Plastic Surgery Inservic...,https://doi.org/10.1016/j.bjps.2023.09.027,The recent introduction of Generative Pre-trai...,Applying GPT-4 to the Plastic Surgery Inservic...,16
3414,ChatGPT for low- and middle-income countries: ...,https://doi.org/10.1016/j.lanwpc.2023.100906,"ChatGPT (OpenAI, San Francisco, CA, USA) has m...",ChatGPT for low- and middle-income countries: ...,2
3415,Interpretable Disease Prediction via Path Reas...,https://doi.org/10.1016/j.knosys.2023.111082,Disease prediction based on patients’ historic...,Interpretable Disease Prediction via Path Reas...,175


In [39]:
# show topics info
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,72,0_education_blessing_reshaping_nursing,"[education, blessing, reshaping, nursing, curs...",[The Utilization of ChatGPT in Reshaping Futur...
1,1,63,1_models_large_language_survey,"[models, large, language, survey, ontology, in...",[Large Language Models: A Comprehensive Survey...
2,2,52,2_visually_blind_trust_vs,"[visually, blind, trust, vs, impaired, charact...","[ChatGPT for Visually Impaired and Blind, Chat..."
3,3,50,3_fish_carp_growth_meal,"[fish, carp, growth, meal, dietary, hematologi...",[Effect of white mulberry (Morus albas l.) On ...
4,4,48,4_higher_computer_students_student,"[higher, computer, students, student, teaching...",[ChatGPT in higher education: Considerations f...
...,...,...,...,...,...
195,195,4,195_neurology_forecasting_2035_series,"[neurology, forecasting, 2035, series, ipsycho...",[Editors' Note: Neurology Education in 2035: T...
196,196,3,196_davinci_chatgptenabled_prototype_robot,"[davinci, chatgptenabled, prototype, robot, ad...",[Response of ChatGPT for Humanoid Robots Role ...
197,197,2,197_ernie_rush_bing_gold,"[ernie, rush, bing, gold, war, chats, determin...",[Efficacy of AI Chats to Determine an Emergenc...
198,198,2,198_virtual_realism_processor_ab1701,"[virtual, realism, processor, ab1701, prenatal...",[AB1701 HOW TO MAKE A VIRTUAL PRESENTATION USI...


In [40]:
topic_model.get_topic_info().to_csv(data_export_path+"/LLM_en_openalex_title_topics_metadata_200.csv",index=False)

In [41]:
# show Topic Positions -> suggestions for further clustering
#topic_model.visualize_topics()
fig_IDM = topic_model.visualize_topics()
fig_IDM.write_html(result_path+"/LLM_en_openalex_title_viz_IDM_200.html")

In [42]:
# visualize Topic Hierarchy -> suggestions for further clustering
fig_hierarchy = topic_model.visualize_hierarchy()
fig_hierarchy.write_html(result_path+"/LLM_en_openalex_title_viz_hierarchy_200.html")

In [43]:
# visualize words
fig_topic_word_scores = topic_model.visualize_barchart(top_n_topics=32)
fig_topic_word_scores.write_html(result_path+"/LLM_en_openalex_title_viz_topic_word_scores_200.html")

In [44]:
# visualize topic similarity -- not useful in our case, maybe include to show not correlated
fig_heatmap = topic_model.visualize_heatmap()
fig_heatmap.write_html(result_path+"/LLM_en_openalex_title_viz_heatmap_200.html")

In [45]:
# visualize documents
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)
# Run the visualization with the original embeddings
topic_model.visualize_documents(docs, embeddings=embeddings)
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# save embeddings (for Tableau)
abstract_df["x"] = reduced_embeddings[:, 0]
abstract_df["y"] = reduced_embeddings[:, 1]


In [46]:
import pickle
with open(result_path+'/LLM_en_openalex_title_embeddings_200.pickle', 'wb') as pkl:
    pickle.dump(embeddings, pkl)

In [47]:
abstract_df.to_csv(data_export_path+"/LLM_en_openalex_title_w_topics_years_positions_200.csv",index=False)